In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Video classification

Video classification is the task of assigning a label or class to an entire video. Videos are expected to have only one class for each video. Video classification models take a video as input and return a prediction about which class the video belongs to. These models can be used to categorize what a video is all about. A real-world application of video classification is action / activity recognition, which is useful for fitness applications. It is also helpful for vision-impaired individuals, especially when they are commuting.

This guide will show you how to:

1. Fine-tune [VideoMAE](https://huggingface.co/docs/transformers/main/en/model_doc/videomae) on a subset of the [UCF101](https://www.crcv.ucf.edu/data/UCF101.php) dataset.
2. Use your fine-tuned model for inference.

<Tip>
The task illustrated in this tutorial is supported by the following model architectures:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[TimeSformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/timesformer), [VideoMAE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/videomae)

<!--End of the generated tip-->

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install -q pytorchvideo transformers evaluate
```

You will use [PyTorchVideo](https://pytorchvideo.org/) (dubbed `pytorchvideo`) to process and prepare the videos.

We encourage you to log in to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to log in:

In [2]:
import os
from huggingface_hub import HfFolder

# Read token from environment variable (more secure)
# You can set this environment variable before running the notebook
# export HUGGINGFACE_TOKEN=your_token_here (Linux/Mac)
# set HUGGINGFACE_TOKEN=your_token_here (Windows)
token = os.getenv("HUGGINGFACE_TOKEN")

if token:
    HfFolder.save_token(token)
    print("Hugging Face token successfully loaded from HUGGINGFACE_TOKEN environment variable.")
else:
    print("HUGGINGFACE_TOKEN environment variable not set. If you want to push models to the Hub, please set this variable before starting Jupyter Lab.")

# Commenting out other options to keep the cell clean
# Option 1: Set token directly in code (not recommended for shared notebooks)
# HfFolder.save_token("your_token_here")

# Option 3: Load token from a file (more secure)
# token_path = "path/to/token.txt"
# if os.path.exists(token_path):
#     with open(token_path, "r") as f:
#         token = f.read().strip()
#     HfFolder.save_token(token)
#     print("Hugging Face token successfully loaded from file.")
# else:
#     print(f"Token file not found at {token_path}")

Hugging Face token successfully loaded from HUGGINGFACE_TOKEN environment variable.


## Load Sports in the Wild dataset

Start by loading a subset of the [SVW dataset](https://cvlab.cse.msu.edu/project-svw.html). This will give you a chance to experiment and make sure everything works before spending more time training on the full dataset.

In [3]:
import os

# Set the path to the local processed dataset folder
dataset_root_path = "processed_dataset"
all_video_file_paths = []

# Read train, validation and test file paths from CSV files
with open(os.path.join(dataset_root_path, "train.csv"), "r") as f:
    train_paths = [line.strip().split()[0] for line in f.readlines()]
    all_video_file_paths.extend([os.path.join(dataset_root_path, path) for path in train_paths])
    
with open(os.path.join(dataset_root_path, "val.csv"), "r") as f:
    val_paths = [line.strip().split()[0] for line in f.readlines()]
    all_video_file_paths.extend([os.path.join(dataset_root_path, path) for path in val_paths])
    
with open(os.path.join(dataset_root_path, "test.csv"), "r") as f:
    test_paths = [line.strip().split()[0] for line in f.readlines()]
    all_video_file_paths.extend([os.path.join(dataset_root_path, path) for path in test_paths])

print(f"Total video files: {len(all_video_file_paths)}")

Total video files: 4206


After the subset has been downloaded, you need to extract the compressed archive:

In [4]:
import os

# Just verify that the videos directory exists
videos_dir = os.path.join(dataset_root_path, "videos")
if os.path.exists(videos_dir):
    print(f"Videos directory exists at {videos_dir}")
    print(f"Number of video files: {len(os.listdir(videos_dir))}")
else:
    print(f"Warning: Videos directory not found at {videos_dir}")

Videos directory exists at processed_dataset\videos
Number of video files: 4206


In [5]:
import os

# Assuming dataset_root_path is defined
# dataset_root_path = "path_to_your_dataset"

# Get labels from CSV files
labels = []

with open(os.path.join(dataset_root_path, "train.csv"), "r") as f:
    for line in f.readlines():
        parts = line.strip().split()
        if len(parts) > 1:
            labels.append(parts[1])

with open(os.path.join(dataset_root_path, "val.csv"), "r") as f:
    for line in f.readlines():
        parts = line.strip().split()
        if len(parts) > 1:
            labels.append(parts[1])
            
with open(os.path.join(dataset_root_path, "test.csv"), "r") as f:
    for line in f.readlines():
        parts = line.strip().split()
        if len(parts) > 1:
            labels.append(parts[1])

# Get unique labels and create mappings
class_labels = sorted(set(labels))
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Unique classes: {len(label2id)}.")
print(f"Class labels: {class_labels}")

Unique classes: 30.
Class labels: ['archery', 'baseball', 'basketball', 'bmx', 'bowling', 'boxing', 'cheerleading', 'discusthrow', 'diving', 'football', 'golf', 'gymnastics', 'hammerthrow', 'highjump', 'hockey', 'hurdling', 'javelin', 'longjump', 'polevault', 'rowing', 'running', 'shotput', 'skating', 'skiing', 'soccer', 'swimming', 'tennis', 'volleyball', 'weight', 'wrestling']


There are 30 unique classes.

## Load a model to fine-tune

Instantiate a video classification model from a pretrained checkpoint and its associated image processor. The model's encoder comes with pre-trained parameters, and the classification head is randomly initialized. The image processor will come in handy when writing the preprocessing pipeline for our dataset.

In [14]:
from transformers import VideoMAEConfig, VideoMAEForVideoClassification, VideoMAEImageProcessor

# Tiny configuration to match the XD Violence tiny notebook
compact_config = VideoMAEConfig(
    num_hidden_layers=4,
    hidden_size=384,
    intermediate_size=1536,
    num_attention_heads=6,
    image_size=224,
    num_frames=16,
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label,
    mask_ratio=0.0,
)

model = VideoMAEForVideoClassification(compact_config)
image_processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model_ckpt = "MCG-NJU/videomae-base-finetuned-kinetics"

While the model is loading, you might notice the following warning:

```bash
Some weights of the model checkpoint at MCG-NJU/videomae-base were not used when initializing VideoMAEForVideoClassification: [..., 'decoder.decoder_layers.1.attention.output.dense.bias', 'decoder.decoder_layers.2.attention.attention.key.weight']
- This IS expected if you are initializing VideoMAEForVideoClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoMAEForVideoClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
```

The warning is telling us we are throwing away some weights (e.g. the weights and bias of the `classifier` layer) and randomly initializing some others (the weights and bias of a new `classifier` layer). This is expected in this case, because we are adding a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

**Note** that [this checkpoint](https://huggingface.co/MCG-NJU/videomae-base-finetuned-kinetics) leads to better performance on this task as the checkpoint was obtained fine-tuning on a similar downstream task having considerable domain overlap. You can check out [this checkpoint](https://huggingface.co/sayakpaul/videomae-base-finetuned-kinetics-finetuned-ucf101-subset) which was obtained by fine-tuning `MCG-NJU/videomae-base-finetuned-kinetics`.

## Prepare the datasets for training

For preprocessing the videos, you will leverage the [PyTorchVideo library](https://pytorchvideo.org/). Start by importing the dependencies we need.

In [15]:
import pytorchvideo.data

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)

For the training dataset transformations, use a combination of uniform temporal subsampling, pixel normalization, random cropping, and random horizontal flipping. For the validation and evaluation dataset transformations, keep the same transformation chain except for random cropping and horizontal flipping. To learn more about the details of these transformations check out the [official documentation of PyTorchVideo](https://pytorchvideo.org).  

Use the `image_processor` associated with the pre-trained model to obtain the following information:

* Image mean and standard deviation with which the video frame pixels will be normalized.
* Spatial resolution to which the video frames will be resized.

Start by defining some constants.

In [16]:
mean = image_processor.image_mean
std = image_processor.image_std
if "shortest_edge" in image_processor.size:
    height = width = image_processor.size["shortest_edge"]
else:
    height = image_processor.size["height"]
    width = image_processor.size["width"]
resize_to = (height, width)

num_frames_to_sample = model.config.num_frames
sample_rate = 4
fps = 30
clip_duration = num_frames_to_sample * sample_rate / fps

Now, define the dataset-specific transformations and the datasets respectively. Starting with the training set:

In [17]:
# Define batch size
batch_size = 8

# Define the transforms for training data
train_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    RandomShortSideScale(min_size=256, max_size=320),
                    RandomCrop(resize_to),
                    RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)

# Helper function to load labeled video paths from a CSV
# This function assumes 'os' module is imported, and 
# 'dataset_root_path' and 'label2id' are defined in the global scope of the notebook.
def load_labeled_video_paths(csv_filename, root_dir_for_csv_paths, label_to_id_map):
    labeled_paths = []
    # Construct the full path to the CSV file
    csv_path = os.path.join(root_dir_for_csv_paths, csv_filename)
    with open(csv_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) >= 2:
                video_path_in_csv = parts[0]  # e.g., "videos/video_000000.mp4"
                label_str = parts[1]
                
                # Construct the full path to the video file
                # root_dir_for_csv_paths is dataset_root_path (e.g., "processed_dataset")
                full_video_path = os.path.join(root_dir_for_csv_paths, video_path_in_csv)
                
                if label_str in label_to_id_map:
                    label_id = label_to_id_map[label_str]
                    labeled_paths.append((full_video_path, {"label": label_id}))
                else:
                    print(f"Warning: Label '{label_str}' not in label2id map for video {full_video_path}. Skipping.")
            elif line.strip(): # Avoid warning for empty lines if any
                print(f"Warning: Malformed line in {csv_path}: '{line.strip()}'")
    return labeled_paths

# Load labeled video paths for training
# dataset_root_path, label2id, clip_duration, and train_transform are assumed to be defined in previous cells.
labeled_video_paths_train = load_labeled_video_paths("train.csv", dataset_root_path, label2id)

# Create LabeledVideoDataset for training
train_dataset = pytorchvideo.data.LabeledVideoDataset(
    labeled_video_paths=labeled_video_paths_train,
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)

The same sequence of workflow can be applied to the validation and evaluation sets:

In [18]:
val_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    Resize(resize_to),
                ]
            ),
        ),
    ]
)

# Load labeled video paths for validation
# dataset_root_path, label2id, clip_duration, and val_transform are assumed to be defined in previous cells.
# The load_labeled_video_paths function is defined in the previous cell.
labeled_video_paths_val = load_labeled_video_paths("val.csv", dataset_root_path, label2id)

val_dataset = pytorchvideo.data.LabeledVideoDataset(
    labeled_video_paths=labeled_video_paths_val,
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)

# Load labeled video paths for test
labeled_video_paths_test = load_labeled_video_paths("test.csv", dataset_root_path, label2id)

test_dataset = pytorchvideo.data.LabeledVideoDataset(
    labeled_video_paths=labeled_video_paths_test,
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform, # Using val_transform for test set as well
)

**Note**: The above dataset pipelines are taken from the [official PyTorchVideo example](https://pytorchvideo.org/docs/tutorial_classification#dataset). We're using the [`pytorchvideo.data.Ucf101()`](https://pytorchvideo.readthedocs.io/en/latest/api/data/data.html#pytorchvideo.data.Ucf101) function because it's tailored for the UCF-101 dataset. Under the hood, it returns a [`pytorchvideo.data.labeled_video_dataset.LabeledVideoDataset`](https://pytorchvideo.readthedocs.io/en/latest/api/data/data.html#pytorchvideo.data.LabeledVideoDataset) object. `LabeledVideoDataset` class is the base class for all things video in the PyTorchVideo dataset. So, if you want to use a custom dataset not supported off-the-shelf by PyTorchVideo, you can extend the `LabeledVideoDataset` class accordingly. Refer to the `data` API [documentation to](https://pytorchvideo.readthedocs.io/en/latest/api/data/data.html) learn more. Also, if your dataset follows a similar structure (as shown above), then using the `pytorchvideo.data.Ucf101()` should work just fine.

You can access the `num_videos` argument to know the number of videos in the dataset.

In [19]:
print(train_dataset.num_videos, val_dataset.num_videos, test_dataset.num_videos)

3364 420 422


## Visualize the preprocessed video for better debugging

In [20]:
import imageio
import numpy as np
from IPython.display import Image

def unnormalize_img(img):
    """Un-normalizes the image pixels."""
    img = (img * std) + mean
    img = (img * 255).astype("uint8")
    return img.clip(0, 255)

def create_gif(video_tensor, filename="sample.gif"):
    """Prepares a GIF from a video tensor.

    The video tensor is expected to have the following shape:
    (num_frames, num_channels, height, width).
    """
    frames = []
    for video_frame in video_tensor:
        frame_unnormalized = unnormalize_img(video_frame.permute(1, 2, 0).numpy())
        frames.append(frame_unnormalized)
    kargs = {"duration": 0.25}
    imageio.mimsave(filename, frames, "GIF", **kargs)
    return filename

def display_gif(video_tensor, gif_name="sample.gif"):
    """Prepares and displays a GIF from a video tensor."""
    video_tensor = video_tensor.permute(1, 0, 2, 3)
    gif_filename = create_gif(video_tensor, gif_name)
    return Image(filename=gif_filename)

sample_video = next(iter(train_dataset))
video_tensor = sample_video["video"]
display_gif(video_tensor)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/sample_gif.gif" alt="Person playing basketball"/>
</div>

## Train the model

Leverage [`Trainer`](https://huggingface.co/docs/transformers/main_classes/trainer) from  🤗 Transformers for training the model. To instantiate a `Trainer`, you need to define the training configuration and an evaluation metric. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to configure the training. It requires an output folder name, which will be used to save the checkpoints of the model. It also helps sync all the information in the model repository on 🤗 Hub.

Most of the training arguments are self-explanatory, but one that is quite important here is `remove_unused_columns=False`. This one will drop any features not used by the model's call function. By default it's `True` because usually it's ideal to drop unused feature columns, making it easier to unpack inputs into the model's call function. But, in this case, you need the unused features ('video' in particular) in order to create `pixel_values` (which is a mandatory key our model expects in its inputs).

In [21]:
from transformers import TrainingArguments, Trainer

model_name = model_ckpt.split("/")[-1]
new_model_name = f"{model_name}-finetuned-sports-videos-in-the-wild"
num_epochs = 20

args = TrainingArguments(
    new_model_name,
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    push_to_hub=True,
    max_steps=(train_dataset.num_videos // batch_size) * num_epochs,
)

The dataset returned by `pytorchvideo.data.Ucf101()` doesn't implement the `__len__` method. As such, we must define `max_steps` when instantiating `TrainingArguments`.

Next, you need to define a function to compute the metrics from the predictions, which will use the `metric` you'll load now. The only preprocessing you have to do is to take the argmax of our predicted logits:

In [22]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Multiclass metrics: macro/weighted

def compute_metrics(eval_pred):
    preds = np.argmax(eval_pred.predictions, axis=1)
    labels = eval_pred.label_ids

    acc = accuracy_score(labels, preds)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro', zero_division=0)
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)

    # Return only scalars to avoid logging/serialization errors
    return {
        'accuracy': acc,
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_macro,
        'weighted_precision': precision_weighted,
        'weighted_recall': recall_weighted,
        'weighted_f1': f1_weighted,
    }

**A note on evaluation**:

In the [VideoMAE paper](https://arxiv.org/abs/2203.12602), the authors use the following evaluation strategy. They evaluate the model on several clips from test videos and apply different crops to those clips and report the aggregate score. However, in the interest of simplicity and brevity, we don't consider that in this tutorial.

Also, define a `collate_fn`, which will be used to batch examples together. Each batch consists of 2 keys, namely `pixel_values` and `labels`.

In [23]:
def collate_fn(examples):
    # permute to (num_frames, num_channels, height, width)
    pixel_values = torch.stack(
        [example["video"].permute(1, 0, 2, 3) for example in examples]
    )
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [24]:
import torch
from transformers import Trainer


def compute_class_weights_from_training_set(labeled_video_paths_train, num_classes):
    counts = torch.zeros(num_classes, dtype=torch.long)
    for _, target in labeled_video_paths_train:
        label_id = target.get("label")
        if label_id is not None and 0 <= label_id < num_classes:
            counts[label_id] += 1
    # Avoid division by zero
    counts = counts.clamp(min=1)
    total = counts.sum().item()
    class_weights = total / (num_classes * counts.float())
    return class_weights


num_classes = len(label2id)
class_weights = compute_class_weights_from_training_set(labeled_video_paths_train, num_classes)


class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Move class weights to the same device as logits
        weights_on_device = self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights_on_device)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


Then you just pass all of this along with the datasets to `Trainer`:

In [25]:
trainer = WeightedLossTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    class_weights=class_weights,
)

C:\Users\order\AppData\Local\Temp\ipykernel_27752\1745477222.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


You might wonder why you passed along the `image_processor` as a tokenizer when you preprocessed the data already. This is only to make sure the image processor configuration file (stored as JSON) will also be uploaded to the repo on the Hub.

Now fine-tune our model by calling the `train` method:

In [26]:
import torch

In [27]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

print(f"Trainer device: {args.device}")
print(f"Model device: {model.device}")

CUDA available: True
Number of CUDA devices: 1
Current CUDA device: 0
Device name: NVIDIA GeForce RTX 3090
Trainer device: cuda:0
Model device: cuda:0


In [28]:
train_results = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: mite_gvg (mitegvg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [29]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/mitegvg/videomae-base-finetuned-kinetics-finetuned-sports-videos-in-the-wild/commit/19b754f7b4726cfc1f235ddbb4582460ad274121', commit_message='End of training', commit_description='', oid='19b754f7b4726cfc1f235ddbb4582460ad274121', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mitegvg/videomae-base-finetuned-kinetics-finetuned-sports-videos-in-the-wild', endpoint='https://huggingface.co', repo_type='model', repo_id='mitegvg/videomae-base-finetuned-kinetics-finetuned-sports-videos-in-the-wild'), pr_revision=None, pr_num=None)

## Inference

Great, now that you have fine-tuned a model, you can use it for inference!

Load a video for inference:

In [30]:
sample_test_video = next(iter(test_dataset))

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/sample_gif_two.gif" alt="Teams playing basketball"/>
</div>

The simplest way to try out your fine-tuned model for inference is to use it in a [`pipeline`](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.VideoClassificationPipeline). Instantiate a `pipeline` for video classification with your model, and pass your video to it:

In [31]:
from transformers import pipeline
import os
import platform

# Load your fine-tuned model for inference
# This should be the directory where your fine-tuned model was saved.
# Based on the training setup, it's "videomae-base-finetuned-xd-violence"
# relative to the notebook's directory.
local_model_directory = "videomae-base-finetuned-sports-videos-in-the-wild"
absolute_model_path = os.path.abspath(local_model_directory)

print(f"Attempting to load model from: {absolute_model_path}")
if not os.path.isdir(absolute_model_path):
    print(f"ERROR: Model directory not found at {absolute_model_path}")
    video_cls = None # Prevent further execution if model not found
else:
    print(f"Model directory found. Initializing pipeline...")
    video_cls = pipeline(task="video-classification", model=absolute_model_path)

if video_cls:
    # Use a local test video for inference
    # Ensure dataset_root_path uses raw string for Windows paths
    # The 'r' prefix is crucial here for Windows paths with backslashes

    test_video_filename = "test0.mp4"

    # Construct the path
    raw_test_video_path = test_video_filename
    
    # Normalize the path to resolve any OS-specific issues (like mixed slashes or redundant separators)
    # and to handle potential escape sequence issues more robustly.
    normalized_test_video_path = os.path.normpath(raw_test_video_path)
    
    print(f"Dataset root path: {dataset_root_path}")
    print(f"Raw video path constructed: {raw_test_video_path}")
    print(f"Normalized video path to be used by av.open: {normalized_test_video_path}")
    
    if not os.path.exists(normalized_test_video_path):
        print(f"ERROR: Video file not found at {normalized_test_video_path}.")
        print(f"Please ensure the file exists at this exact path.")
        print(f"Expected directory for videos: {os.path.normpath(os.path.join(dataset_root_path, 'videos'))}")
    else:
        print(f"Video file found at {normalized_test_video_path}. Proceeding with classification.")
        try:
            result = video_cls(normalized_test_video_path)
            print("Inference result:")
            print(result)
        except Exception as e:
            print(f"An error occurred during video classification: {e}")
            import traceback
            traceback.print_exc()
else:
    print("Skipping inference due to model loading error.")

Attempting to load model from: D:\BIRKBECK\REPOS\videomae-base-finetuned-sports-in-the-wild\videomae-base-finetuned-sports-videos-in-the-wild
Model directory found. Initializing pipeline...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


Dataset root path: processed_dataset
Raw video path constructed: test0.mp4
Normalized video path to be used by av.open: test0.mp4
Video file found at test0.mp4. Proceeding with classification.
Inference result:
[{'score': 0.6231321692466736, 'label': 'hockey'}, {'score': 0.12232962995767593, 'label': 'bmx'}, {'score': 0.08188006281852722, 'label': 'archery'}, {'score': 0.022075997665524483, 'label': 'hammerthrow'}, {'score': 0.017382944002747536, 'label': 'football'}]


You can also manually replicate the results of the `pipeline` if you'd like.

# Evaluate on Full Test Set
This section evaluates the fine-tuned model on all videos specified in the `test.csv` file.
It calculates and reports both Top-1 and Top-5 accuracy.

In [34]:
import os
from transformers import pipeline
import torch  # For checking device
import time
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Starting evaluation on the full test set...")

# Define paths (relative to the notebook location)
dataset_root_path = "processed_dataset"
test_csv_filename = "test.csv"
test_csv_path = os.path.join(dataset_root_path, test_csv_filename)

local_model_directory = "videomae-base-finetuned-kinetics-finetuned-sports-videos-in-the-wild"
absolute_model_path = os.path.abspath(local_model_directory)

# Function to load test data (video paths and true labels)
def load_test_data_from_csv(csv_file_path, data_root_path):
    test_samples = []
    if not os.path.exists(csv_file_path):
        print(f"ERROR: Test CSV file not found at {csv_file_path}")
        return test_samples

    with open(csv_file_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) >= 2:
                relative_video_path = parts[0]  # e.g., "videos/video_000000.mp4"
                true_label_str = parts[1]

                full_video_path = os.path.normpath(os.path.join(data_root_path, relative_video_path))
                test_samples.append((full_video_path, true_label_str))
            elif line.strip():
                print(f"Warning: Malformed line in {csv_file_path}: '{line.strip()}'")
    print(f"Loaded {len(test_samples)} samples from {csv_file_path}")
    return test_samples

# Initialize the video classification pipeline
video_cls = None
print(f"Attempting to load model from: {absolute_model_path}")
if not os.path.isdir(absolute_model_path):
    print(f"ERROR: Model directory not found at {absolute_model_path}")
else:
    print(f"Model directory found. Initializing pipeline...")
    try:
        video_cls = pipeline(
            task="video-classification",
            model=absolute_model_path,
            device=0 if torch.cuda.is_available() else -1  # Use GPU if available
        )
        print(f"Pipeline initialized. Using device: {'cuda:0' if torch.cuda.is_available() else 'cpu'}")
    except Exception as e:
        print(f"Error initializing pipeline: {e}")

if video_cls:
    # Load test data
    test_data = load_test_data_from_csv(test_csv_path, dataset_root_path)

    if test_data:
        total_videos_processed = 0
        true_labels = []
        predicted_labels = []
        top1_correct_predictions = 0
        top5_correct_predictions = 0
        inference_times = []

        print(f"\nStarting inference on {len(test_data)} test videos...")
        for i, (video_path, true_label) in enumerate(test_data):
            if not os.path.exists(video_path):
                print(f"Warning: Video file not found at {video_path}. Skipping.")
                continue

            try:
                start_time = time.time()
                raw_results = video_cls(video_path)
                end_time = time.time()
                inference_times.append(end_time - start_time)
                total_videos_processed += 1

                if not raw_results:
                    print(f"Warning: No results returned for video {video_path}. Skipping.")
                    continue

                # Extract top-5 predicted labels
                top5_results = raw_results[:5]
                predicted_labels_top5 = [res['label'].split('-')[0] for res in top5_results]
                predicted_label_top1 = predicted_labels_top5[0] if predicted_labels_top5 else None

                # Accumulate for per-class report
                if predicted_label_top1 is not None:
                    predicted_labels.append(predicted_label_top1)
                    true_labels.append(true_label)

                # Top-k accuracy
                if predicted_label_top1 == true_label:
                    top1_correct_predictions += 1
                if true_label in predicted_labels_top5:
                    top5_correct_predictions += 1

                if (i + 1) % 10 == 0 or (i + 1) == len(test_data):
                    print(f"  Processed {i + 1}/{len(test_data)} videos...")

            except Exception as e:
                print(f"An error occurred during processing of {video_path}: {e}")

        if total_videos_processed > 0:
            # Accuracies
            top1_accuracy = (top1_correct_predictions / total_videos_processed) * 100
            top5_accuracy = (top5_correct_predictions / total_videos_processed) * 100

            # Timing
            avg_inference_time = sum(inference_times) / len(inference_times)
            fps = 1.0 / avg_inference_time if avg_inference_time > 0 else float('inf')

            print("\n--- Evaluation Complete ---")
            print(f"Total videos processed: {total_videos_processed}")
            print(f"Top-1 Correct Predictions: {top1_correct_predictions}")
            print(f"Top-5 Correct Predictions: {top5_correct_predictions}")
            print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
            print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
            print(f"Average inference time per video: {avg_inference_time:.3f} seconds ({fps:.2f} videos/sec)")

            # Multiclass report/confusion matrix (only for successfully accumulated items)
            if len(predicted_labels) == len(true_labels) and len(true_labels) > 0:
                print("\nDetailed Multiclass Classification Report:")
                print(classification_report(true_labels, predicted_labels, labels=list(label2id.keys())))

                print("\nConfusion Matrix:")
                cm = confusion_matrix(true_labels, predicted_labels, labels=list(label2id.keys()))
                print(cm)
        else:
            print("\n--- Evaluation Complete ---")
            print("No videos were processed successfully.")
    else:
        print("No test data loaded. Cannot perform evaluation.")
else:
    print("Video classification pipeline not initialized. Cannot perform evaluation.")

Starting evaluation on the full test set...
Attempting to load model from: D:\BIRKBECK\REPOS\videomae-base-finetuned-sports-in-the-wild\videomae-base-finetuned-kinetics-finetuned-sports-videos-in-the-wild
Model directory found. Initializing pipeline...


Device set to use cuda:0


Pipeline initialized. Using device: cuda:0
Loaded 422 samples from processed_dataset\test.csv

Starting inference on 422 test videos...
  Processed 10/422 videos...
  Processed 20/422 videos...
  Processed 30/422 videos...
  Processed 40/422 videos...
  Processed 50/422 videos...
  Processed 60/422 videos...
  Processed 70/422 videos...
  Processed 80/422 videos...
  Processed 90/422 videos...
  Processed 100/422 videos...
  Processed 110/422 videos...
  Processed 120/422 videos...
  Processed 130/422 videos...
  Processed 140/422 videos...
  Processed 150/422 videos...
  Processed 160/422 videos...
  Processed 170/422 videos...
  Processed 180/422 videos...
  Processed 190/422 videos...
  Processed 200/422 videos...
  Processed 210/422 videos...
  Processed 220/422 videos...
  Processed 230/422 videos...
  Processed 240/422 videos...
  Processed 250/422 videos...
  Processed 260/422 videos...
  Processed 270/422 videos...
  Processed 280/422 videos...
  Processed 290/422 videos...
  P

C:\Users\order\.conda\envs\videomae\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\order\.conda\envs\videomae\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\order\.conda\envs\videomae\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

     archery       0.38      0.62      0.47        13
    baseball       0.43      0.50      0.46        18
  basketball       0.39      0.58      0.47        12
         bmx       0.43      0.27      0.33        11
     bowling       0.29      0.50      0.37        10
      boxing       0.14      0.09      0.11        11
cheerleading       0.40      0.53      0.45        19
 discusthrow       0.17      0.25      0.20         4
      diving       0.44      0.73      0.55        11
    football       0.44      0.60      0.51        20
        golf       0.35      0.64      0.45        11
  gymnastics       0.25      0.24      0.24        17
 hammerthrow       0.12      0.12      0.12        16
    highjump       0.25      0.19      0.21        16
      hockey       0.53      0.71      0.61        14
    hurdling       0.10      0.07      0.08        14
     javelin       0.60      0.30      0.40        10
    longjump       0.27    